# IMPORT

In [1]:
try:
    import google.colab # type: ignore
    colab = True
except:
    colab = False

if colab:
    !git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

In [2]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
 
# Our files
if colab:
    sys.path.append('/content/eai-project/training')
    sys.path.append('/content/eai-project/preprocessing')
    sys.path.append('/content/eai-project/dataset')
    sys.path.append('/content/eai-project/utils')
    sys.path.append('/content/eai-project/models')
    sys.path.append('/content/eai-project/testing')
    from train import train
    from preprocessing import *
    from thermal_dataset import thermal_dataset 
    from utils import *
    from complete_model import complete_model 
    from lstm_only import lstm_only
    from testing import test
    prefix = '/content'
        
else:
    from training.train import train
    from preprocessing.preprocessing import *
    from dataset.thermal_dataset import thermal_dataset
    from utils.csv_utils import *
    from models.complete_model import complete_model
    from models.complete_model_autoencoder import complete_model_autoencoder
    from models.lstm_only import lstm_only
    from testing.test import test
    from blocks import mlp, linear, rnn, lstm, cnn
    prefix = '.'
    
pd.set_option('display.max_columns', None)

In [3]:
autoencoder_train = False # to train only the autoencoder

hyper_parameters = {'num_epochs': 100,
                    'batch_size': 256,
                    'hidden_dim' : 3,
                    'lr': 0.001,
                    'mode': 'auto-weighted', #average, mlp, linear, auto-weighted
                    'extractor_type': 'conv', #lstm (if you want to use the lstm autoencoder), conv (if you want to use the convolutional autoencoder)
                    'ensemble': True,
                    'timesteps': 200,
                    'window_size':30,
                    'norm': 'Std', # Not (Raw Data), Minmax (MinMax Scaling), Std (Standard Scaling)
                    'file':1,
                    'weight_decay': 0,
                    'pretrain':False,
                    'heterogeneous': True,
                    'lr_multipliers_extractor': 0.1,
                    'lr_multipliers_ensemble': {
                        mlp: 10,
                        linear: 10,
                        rnn: 1,
                        lstm: 1,
                        cnn: 1
                    }

                        }

# Parameters of the convolutional autoencoder
conv_autoencoder_dict = {'in_kern_out': [[4, 5, 5], [5, 5, 6],[6, 5, 7]], # List of hyperparam of autoencoder [[in_channels, kernel_size, out_channels], ...]
                    'pooling_kernel_size': 2, # how much big is the kernel of the pooling (i.e. 2 means halving the dimension each layer)
                    'padding': 'same', # 'same', 'full', 'valid'
                    'pooling': 'max', # 'max' for Max Pooling and 'avg' for Average Pooling 
                    'scale_factor': 2, # upsample scale_factor, 2 means double the dimension each layer
                    'upsample_mode': 'linear', # mode of the upsampling 'linear', 'nearest'
                    'dropout': 0.0
 }

lstm_autoencoder_dict = {'in_hidd': [[4, 3], [3,2]], # List of hyperparam of autoencoder [[input_dim, hidden_dim], ...]
                        'dropout': 0.75
 }


if hyper_parameters['extractor_type'] == 'conv':
    feature_dim = int(( hyper_parameters['timesteps'] / (2*(len(conv_autoencoder_dict['in_kern_out'])-1))*conv_autoencoder_dict['in_kern_out'][-1][-1]) /2)
elif hyper_parameters['extractor_type'] == 'lstm':
    feature_dim = hyper_parameters['timesteps']*lstm_autoencoder_dict['in_hidd'][-1][-1] 
if hyper_parameters['heterogeneous']:
    feature_dim += 3
    

# Definition of the model (You can decomment to include more models)
ensemble_model = {
    'mlp': [{'layer_dim_list': [ feature_dim,int(feature_dim/1.5),int(feature_dim/3),int(feature_dim/5),int(feature_dim/8),int(feature_dim/10),3]}],
            #{'layer_dim_list': [ feature_dim,int(feature_dim*1.5),3]}], 
    'ARIMA': [{'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1}], 
    #'linear_regressor': [{'in_features': feature_dim, 'out_features': 3, 'bias':False}],
    'lstm': [{'feature_dim':3, 'input_dim':feature_dim, 'num_layers':1}],
    #'rnn': [{'feature_dim':3, 'input_dim':feature_dim, 'num_layers':1}],
    #'cnn': [{'input_dim':hyper_parameters['hidden_dim'],
    #            'hidden_dim_list': [ int(feature_dim/10),int(feature_dim/20),int(feature_dim/30)],
    #            'output_dim': 3,
    #            'kernel_size_list':[2,4,2]
    #            }]
}

In [4]:
skip = False
test_flag = True

if os.path.exists(prefix+'/data/X'+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy') and os.path.exists(prefix+'/data/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy'):
    print("Data already exists!")
    print(prefix+'/data/X'+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy')
    print(prefix+'/data/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy')
    skip = True

Data already exists!
./data/Xtraining1Std.npy
./data/Y30training1Std.npy


**Reproducibility and Device Setting**

In [5]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# DATASET

In [6]:
link_zipped_csv = 'https://drive.google.com/file/d/1MssQF4pI_rZqiiDBP4XaLTT1ZaN6ykLm/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)


CSV file already downloaded!
CSV file already unzipped!


In [7]:
if not skip:

    path = '/content/csv/thermal_drift_features_lab_05_02.csv'

    # Read all the CSV files containing the Temperatures
    features_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_02.csv'))
    features_2 = pd.read_csv(os.path.join(prefix, 'csv/thermal_drift_features_lab_05_03.csv'))
    features_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_04.csv'))
    features_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_05.csv'))
    features_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_06.csv'))

    # Read all the CSV files containing the X1, Y1, Z1 
    targets_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_02.csv'))
    targets_2 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_03.csv'))
    targets_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_04.csv'))
    targets_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_05.csv'))
    targets_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_06.csv'))

    features = [features_1, features_2, features_3, features_4, features_5]
    targets = [targets_1,targets_2,targets_3,targets_4,targets_5]

In [8]:
if not skip:
    for feature, target in zip(features, targets):
        feature.drop([
            "name", "tags",
            "2\"Tray1 Vacuum Sensor", "2\"Tray2 Vacuum Sensor", "2\"Tray3 Vacuum Sensor",
            "Avg Oven Temperature", "Chuck Temp [Cdeg]", "Chuck Temp2 [Cdeg]",
            "Chuck1 Vacuum Sensor", "Contrast", "Device State",
            "Dispenser1 Pressure Sensor", "Machine Room Temp", "Main Air", "Main Vacuum",
            "Oven Temperature", "PE_Rx", "PE_Ry", "PE_Rz", "PE_X1", "PE_Y1", "PE_Z1",
            "PUT1 Flow Sensor", "PUT2 Flow Sensor1", "PUT2 Flow Sensor2",
            "PUT2 Flow Sensor3", "PUT2 Flow Sensor4", "PUT2 Flow Sensor5",
            "Photodiode", "Pixel Power", "Preciser1 Vacuum Sensor",
            "Tec FIB1 Holder", "Tec FIB1 Plate", "Tec FIB2 Holder", "Tec FIB2 Plate",
            "Torque11","Torque2","Torque3","Torque4","Torque5","Torque6"
        ], axis=1, inplace=True)
        if 'name' in target.keys() and 'tags' in target.keys():

            target.drop(['name', 'tags'], axis=1, inplace=True)
            
            


In [9]:
if not skip:
    print(features[1]) # Print the features_1 table

In [10]:
if not skip:

    print(targets[1]) # Print the target_1

In [11]:
if not skip:
    # Put X1, Y1, Z1 on the same row of X1 eliminating the NAN values
    fixed_targets = [] # Create a list of target in which we put X1, Y1, Z1 in the same row
    for target in targets:
        fixed_targets.append(transform_dataframe(target)) # iterate over target_1,2,3 ... and append in fixed_targets
        
        

In [12]:
if not skip:

    print(fixed_targets[1]) # Print the fixed_target_1

In [13]:
if not skip:
    # Merge of targets with features in one single dataframe
    complete_numbers_list = [] # List of the table with columns that are numbers (0,1,2..) in which we unify both features and targets merging on closest time row
    for fixed_target, feature in zip(fixed_targets, features):
        complete_numbers_list.append(merge_on_closest_time(fixed_target.reset_index(), feature.reset_index()))

    trainig_number_list = []
    testing_number_list = []
    for i in range(len(complete_numbers_list)):
        part_numbers_list = complete_numbers_list[:i] + complete_numbers_list[i+1:]
        trainig_number_list.append(pd.concat(part_numbers_list))
        testing_number_list.append(complete_numbers_list[i])
        
    complete_numbers_dataframe = pd.concat(complete_numbers_list)
    


In [14]:
if not skip: 
    print(complete_numbers_list) # Print of one example

In [15]:
if not skip:

    complete = complete_numbers_dataframe.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'time_2',
        7: 'Temp1',
        8: 'Temp2',
        9: 'Temp3',
        10: 'Temp4'
        })
    complete.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
    training_list = []
    testing_list = []
    for training, testing in zip(trainig_number_list, testing_number_list):
        training_tmp = training.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        training_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        training_list.append(training_tmp)
        
        testing_tmp = testing.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        testing_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        testing_list.append(testing_tmp)
        

In [16]:
if not skip:

    print(training_list)

In [17]:
if not skip:

    # Save the features and targets in file npy
    for i, (testing, training) in enumerate(zip(testing_list, training_list)):

        # Transform the training and test data in float
        training.astype(float) 
        testing.astype(float) 

        # Take from dataframe the values of the columns of temperatures and positions saving into smallest dataframe of training/test
        X_train = training[['Temp1','Temp2', 'Temp3', 'Temp4']] 
        Y_train = training[['X1', 'Y1', 'Z1']] 
        X_test = testing[['Temp1','Temp2', 'Temp3', 'Temp4']] 
        Y_test = testing[['X1', 'Y1', 'Z1']] 

        # Transform the X, Y from dataframe in numpy array both for test and train
        X_train = X_train.values.astype(np.float32) 
        Y_train = Y_train.values.astype(np.float32) 
        X_test = X_test.values.astype(np.float32) 
        Y_test = Y_test.values.astype(np.float32) 

        # Do the gradient of the positions both for test and train
        Y_train = my_gradient(Y_train, window_size=hyper_parameters['window_size']) 
        Y_test = my_gradient(Y_test, window_size=hyper_parameters['window_size']) 
 

        # Apply the scaling
        if hyper_parameters['norm'] != 'Not':
            if hyper_parameters['norm'] == 'Minmax':
                scaler = MinMaxScaler()
            elif hyper_parameters['norm'] == 'Std':
                scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)
            X_test = scaler.fit_transform(X_test)
            Y_train = scaler.fit_transform(Y_train)
            Y_test = scaler.fit_transform(Y_test)
        else:
            print("Using Raw Data!")
    
        
        # Save the features and targets in file npy
        np.save(prefix+'/data'+'/X'+'training'+str(i)+str(hyper_parameters['norm'])+'.npy',X_train)  
        np.save(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'training'+str(i)+str(hyper_parameters['norm'])+'.npy',Y_train)
        np.save(prefix+'/data'+'/X'+'testing'+str(i)+str(hyper_parameters['norm'])+'.npy',X_test)
        np.save(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'testing'+str(i)+str(hyper_parameters['norm'])+'.npy',Y_test)


In [18]:
if not skip:

    targets_1['time'] = pd.to_datetime(targets_1['time'], unit='ns')

    # Reshape the DataFrame using melt()
    targets_melted = targets_1.reset_index().melt(id_vars=['time'], value_vars=['X1', 'Y1', 'Z1'], var_name='variable', value_name='value')

    # Drop rows where 'value' is NaN (to keep only the non-null entries)
    targets_melted = targets_melted.dropna(subset=['value'])

    # Plot the data
    plt.figure(figsize=(10, 6))
    for var in targets_melted['variable'].unique():
        # Filter data for each variable and plot
        temp_df = targets_melted[targets_melted['variable'] == var]
        plt.plot(temp_df['time'], temp_df['value'], label=var)

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of X1, Y1, Z1 over Time')
    plt.legend(title='Variable')

    # Show the plot
    plt.show()

    #targets.plot(y='X1',x='time')

In [19]:
if not skip:

    # Convert 'time' to datetime (nanoseconds to datetime)
    features_1['time'] = pd.to_datetime(features_1['time'], unit='ns')

    # Set 'time' as the index
    features_1.set_index('time', inplace=True)

    # Optionally, you can plot X1, Y1, Z1 directly
    features_1[['Temp Sensor 1', 'Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4', ]].plot()

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of Temp Sensor 1, Temp Sensor 2, Temp Sensor 3, Temp Sensor 4 over Time')

    # Show the plot
    plt.show()

In [20]:
X = np.load(prefix+'/data'+'/X'+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy')
Y = np.load(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+str(hyper_parameters['norm'])+'.npy')

print(X)
print(Y)

splitPerc = [0.7,0.3]
splitted_X = split(X, splitPerc)
splitted_Y = split(Y, splitPerc)

[[ 8.2750964e-01  3.6703822e-01  5.5890518e-01  9.1104820e-02]
 [ 8.2750964e-01  6.2521166e-01  5.1233166e-01  1.4015545e-01]
 [ 8.2750964e-01  8.8338506e-01  5.5890518e-01  1.4015545e-01]
 ...
 [ 3.4184408e-01  7.9732728e-01  2.2581613e-05 -4.4845200e-01]
 [ 3.4184408e-01  7.1126944e-01 -4.6550971e-02 -3.5035077e-01]
 [ 3.4184408e-01  7.1126944e-01  2.2581613e-05 -4.4845200e-01]]
[[1.62778128 0.52317057 3.71087109]
 [1.21107403 0.58705623 4.16294433]
 [0.85172156 0.63696138 4.30145188]
 ...
 [1.48545645 0.79269374 0.93879646]
 [1.46527607 0.79906461 0.99458422]
 [1.45607098 0.83136138 0.98304193]]
0
15880
0
15859


# MODEL

In [21]:
datasetTrain = thermal_dataset((splitted_X[0],splitted_Y[0]), hyper_parameters['timesteps'], device)
datasetVal = thermal_dataset((splitted_X[1],splitted_Y[1]), hyper_parameters['timesteps'], device)

In [22]:
# Training, Test and Validation Dataloader initialization
loaderTrain = DataLoader(datasetTrain, shuffle=True, batch_size=hyper_parameters['batch_size'])
loaderVal = DataLoader(datasetVal, shuffle=True, batch_size=hyper_parameters['batch_size'])

# Model Initialization (True if you want to use the Ensemble model, False in you want to use a single LSTM model)
if hyper_parameters['ensemble'] == False:
    model = lstm_only(hidden_dim=hyper_parameters['hidden_dim'],
                      timesteps=hyper_parameters['timesteps'] ,
                      input_dim=4, 
                      output_dim=3).to(device)
    
elif hyper_parameters['ensemble'] == True:
    if hyper_parameters['extractor_type'] == 'conv':
        model = complete_model_autoencoder(ensemble_model, 
                                           device, 
                                           autoencoder_dim=conv_autoencoder_dict['in_kern_out'], 
                                           pooling_kernel_size=conv_autoencoder_dict['pooling_kernel_size'], 
                                           padding=conv_autoencoder_dict['padding'], 
                                           pooling=conv_autoencoder_dict['pooling'], 
                                           scale_factor = conv_autoencoder_dict['scale_factor'], 
                                           upsample_mode=conv_autoencoder_dict['upsample_mode'], 
                                           dropout=conv_autoencoder_dict['dropout'],
                                           mode=hyper_parameters['mode'],
                                           heterogeneous=hyper_parameters['heterogeneous'],
                                           timesteps=hyper_parameters['timesteps']
                                           ).to(device)
    
    elif hyper_parameters['extractor_type'] == 'lstm':
        model = complete_model_autoencoder(
                                    model_dict=ensemble_model, 
                                    device=device, 
                                    timesteps=hyper_parameters['timesteps'],
                                    autoencoder_dim=lstm_autoencoder_dict['in_hidd'], 
                                    dropout=lstm_autoencoder_dict['dropout'],
                                    extractor_type=hyper_parameters['extractor_type'],
                                    heterogeneous=hyper_parameters['heterogeneous']
                                    ).to(device)
    else:
        model = complete_model(
                               hidden_dim=hyper_parameters['hidden_dim'], 
                               input_dim=4, 
                               model_dict=ensemble_model, 
                               device=device, 
                               mode=hyper_parameters['mode'], 
                               extractor_type=hyper_parameters['extractor_type']
                               ).to(device)

# Definition of the optimizer and loss function

if hyper_parameters['ensemble'] == False:
    optimizer = optim.Adam(model.parameters(), lr=hyper_parameters['lr'])
elif hyper_parameters['ensemble'] == True:


    models, arima, rnn_ = model.get_models()
    list_models = models + arima + rnn_ 
    optimizer = [optim.Adam(model.extractor.parameters(), lr=hyper_parameters['lr']*hyper_parameters['lr_multipliers_extractor'])]
    for sub_model in list_models: 
        for model_type, multiplier in hyper_parameters['lr_multipliers_ensemble'].items():
            if isinstance(sub_model, model_type):
                optimizer.append(optim.Adam(model.parameters(), lr=hyper_parameters['lr']*multiplier, weight_decay=hyper_parameters['weight_decay']))
                break
    print(len(optimizer))

hyper_parameters['lr_multipliers_ensemble'] = {
        'mlp': hyper_parameters['lr_multipliers_ensemble'][mlp],
        'linear': hyper_parameters['lr_multipliers_ensemble'][linear],
        'rnn': hyper_parameters['lr_multipliers_ensemble'][rnn],
        'lstm': hyper_parameters['lr_multipliers_ensemble'][lstm],
        'cnn': hyper_parameters['lr_multipliers_ensemble'][cnn]
    }


loss_fn = nn.MSELoss()
if hyper_parameters['ensemble'] == False:
    scheduler = ExponentialLR(optimizer, gamma=0.85)
elif hyper_parameters['ensemble'] == True:
    scheduler = [ExponentialLR(opti, gamma=0.85) for opti in optimizer]
model

TypeError: __init__() missing 1 required positional argument: 'timesteps'

In [ ]:
if hyper_parameters['pretrain']:
    model.load('./results/training_2025-02-25_17-35/autoencoder.pt',autoencoder=hyper_parameters['pretrain'])

train(
    num_epochs=hyper_parameters['num_epochs'],
    loss_fn=loss_fn,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    training_dataloader=loaderTrain,
    validation_dataloader=loaderVal,
    hyperparams=hyper_parameters,
    model_dict = ensemble_model,
    autoencoder_dict=lstm_autoencoder_dict,
    complete=hyper_parameters['ensemble'],
    autoencoder_train=autoencoder_train
)
